# Stock Valuation 2

This project aims to create a portfolio of stocks to buy. The priority is not to pick winning stocks, but to avoid picking losing stocks. I will aim to do this by assessing the value, growth, income and momentum of the companies in the US and UK stock market, and make a portfolio of these companies. 

# Stock Analysis

### Information Technology Sector

In [5]:
import pandas as pd
import yfinance as yf
import numpy as np
import logging

# Configure logging
logging.basicConfig(level=logging.ERROR, format='%(levelname)s:%(message)s')

# Load the CSV file containing all US companies
df = pd.read_csv('us_stock_data.csv')

# Define the function to calculate alpha based on the notebook's logic
def calculate_alpha(stock_returns, market_returns, risk_free_rate=0.03):
    # Align the series by their dates
    returns = pd.DataFrame({'stock': stock_returns, 'market': market_returns}).dropna()
    
    # Ensure there are enough data points to calculate covariance and variance
    if len(returns) < 2:
        return np.nan
    
    # Calculate the excess returns
    returns['excess_stock'] = returns['stock'] - risk_free_rate / 252
    returns['excess_market'] = returns['market'] - risk_free_rate / 252
    
    # Calculate the covariance and variance
    covariance = np.cov(returns['excess_stock'], returns['excess_market'])[0, 1]
    variance = np.var(returns['excess_market'])
    
    if variance == 0:
        return np.nan
    
    # Calculate beta
    beta = covariance / variance
    
    # Calculate alpha
    alpha = (returns['excess_stock'].mean() - beta * returns['excess_market'].mean()) * 252
    return alpha

# Function to fetch P/E ratio, Alpha, Volatility, Market Cap, and price appreciation for a list of tickers
def fetch_metrics(tickers):
    metrics = {
        'symbol': [],
        'P/E Ratio': [],
        'Alpha (%)': [],
        'Volatility (%)': [],
        'Market Cap (B)': [],
        '6 Month Appreciation (%)': []
    }

    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            hist = stock.history(period="10y")
            
            if hist.empty:
                logging.error(f"{ticker}: No historical data found.")
                continue
            
            # P/E Ratio
            pe_ratio = stock.info.get('forwardPE', None)
            if pe_ratio is None or pe_ratio <= 0 or pe_ratio >= 25:
                continue
            
            # Calculate 6 month appreciation
            if len(hist) < 126:
                logging.error(f"{ticker}: Not enough data for 6 months calculation.")
                continue
            
            six_month_appreciation = ((hist['Close'][-1] - hist['Close'][-126]) / hist['Close'][-126]) * 100
            if six_month_appreciation <= 0:
                continue
            
            # Calculate Alpha
            stock_returns = hist['Close'].pct_change().dropna()
            market_returns = yf.Ticker('^GSPC').history(period="10y")['Close'].pct_change().dropna()
            
            if market_returns.empty:
                logging.error(f"{ticker}: No market data found for S&P 500.")
                continue
            
            alpha = calculate_alpha(stock_returns, market_returns) * 100
            
            # Calculate Volatility
            volatility = stock_returns.std() * (252 ** 0.5) * 100
            
            # Market Cap in billions
            market_cap = stock.info.get('marketCap', None)
            if market_cap is None:
                continue
            market_cap_b = market_cap / 1e9

            metrics['symbol'].append(ticker)
            metrics['P/E Ratio'].append(pe_ratio)
            metrics['Alpha (%)'].append(alpha)
            metrics['Volatility (%)'].append(volatility)
            metrics['Market Cap (B)'].append(market_cap_b)
            metrics['6 Month Appreciation (%)'].append(six_month_appreciation)
        
        except Exception as e:
            logging.error(f"Error processing {ticker}: {e}")

    return pd.DataFrame(metrics)

# Use tickers from the loaded dataframe
tickers = df['symbol'].unique()

# Fetch the metrics
stock_metrics_df = fetch_metrics(tickers)  # Limiting to first 50 tickers for example purposes

# Order the stocks based on Market Cap
stock_metrics_df = stock_metrics_df.sort_values(by='Market Cap (Billion $)', ascending=False)

# Display the table
print(stock_metrics_df)

# Optionally, save the table to a CSV file
stock_metrics_df.to_csv('stock_metrics.csv', index=False)

ERROR:Error processing AEYE: '<=' not supported between instances of 'str' and 'int'
ERROR:Error processing AHI: '<=' not supported between instances of 'str' and 'int'
ERROR:Error processing ALTG: '<=' not supported between instances of 'str' and 'int'
ERROR:Error processing ARAY: '<=' not supported between instances of 'str' and 'int'
ERROR:AS: Not enough data for 6 months calculation.
ERROR:Error processing AVAH: '<=' not supported between instances of 'str' and 'int'
ERROR:Error processing BCOV: '<=' not supported between instances of 'str' and 'int'
ERROR:BFX: No data found, symbol may be delisted
ERROR:BFX: No historical data found.
ERROR:BODY: No data found, symbol may be delisted
ERROR:BODY: No historical data found.
ERROR:BPTS: No data found, symbol may be delisted
ERROR:BPTS: No historical data found.
ERROR:BTSG: Not enough data for 6 months calculation.
ERROR:CTRI: Not enough data for 6 months calculation.
ERROR:Error processing DDD: '<=' not supported between instances of '

KeyError: 'Market Cap (Billion $)'